# Pandas (Parte 2)

En la última sesión vimos una introducción a Pandas. 
Las taraeas que aprendimos a hacer fueron:

- Crear y leer una base de datos. 
- Realizar los primeros análisis descriptivos de una base de datos. 
- Seleccionar información (por columnas  y filas). 
- Tabular información 
- Filtrar información 

### Ordenando valores 

A veces queremos ordenar nuestro dataframe de acuerdo a criterios específicos

## Limpieza de datos

In [ ]:
import pandas as pd

In [ ]:
locacion_datos = "https://otorongo.club/2021/json/ingresos/"

cong = pd.read_json(locacion_datos)

In [ ]:

## Ordenando de menor a mayor
cong.sort_values("total_ingreso", inplace = True)
#cong.head(4)

In [ ]:
## Ordenando de mayor a menor
cong.sort_values("total_ingreso",ascending = False,  inplace = True)
#cong.head(3)

In [ ]:
## Ordenando con más de un criterio. 
cong.sort_values(['partido', 'total_ingreso'],ascending = False,  inplace = True)
#cong.head(10)

### Cambiando de nombre a nuestras columnas
Muchas veces necesitamos cambiar de nombre a nuestras columnas. Podemos hacerlo directamente: 

In [ ]:
cong2 = cong.copy(deep = True)

In [ ]:
cong2.columns

In [ ]:
cong2.columns = ['nombre_congresista', "dni_cong"] + list(cong2.columns[2:]) ## Esta lista al final 

Otra forma de cambiar nombres es definir un diccionario donde se da el ```nombre_viejo: nombre_nuevo ```

In [ ]:
nuevos_nombres = {'nombre_congresista': 'nombre', 'dni_cong' : 'dni'}

In [ ]:
cong2.rename(columns = nuevos_nombres, inplace = True)

### Cambiando el valor de las variables
Una forma práctica de cambiar los valores de nuestras variables, es usar un diccionario: 

In [ ]:
cong2['partido_nuevo'] = cong2['partido']

cong2.replace({'partido_nuevo': {'ALIANZA PARA EL PROGRESO': 'FUERZA POPULAR',
                                 'JUNTOS POR EL PERU': 'PARTIDO POLITICO NACIONAL PERU LIBRE'}}, inplace = True)

También podemos usar .loc para reemplazar valores:

In [ ]:
cong2.loc[cong2['partido'] == 'ALIANZA PARA EL PROGRESO', 'partido'] = 'FUERZA POPULAR'

Tambien podemos usar el comando ``` isin``` :

In [ ]:
coalicion = ['ALIANZA PARA EL PROGRESO','FUERZA POPULAR', 'VICTORIA NACIONAL']
cong2['nueva_coalicion'] = 0
cong2.loc[cong2["partido"].isin(coalicion), "nueva_coalicion"] = 1

In [ ]:
def clasifica_ingreso(x):
    y = ""
    if x <= 40_000:
        y = "bajo"
    elif ((x > 40_000) and (x < 100_000)):
        y = "medio"
    else:
        y = "alto"
    return y
        

In [ ]:
cong2['ingreso_escalonado'] = cong2['total_ingreso'].apply(lambda x:  clasifica_ingreso(x))

In [ ]:
cong2['ingreso_escalonado'].value_counts()

### Cómo crear variables nuevas

En este caso estoy utilizando indistintamente "variable" de "columna"

In [ ]:
cong['nueva columna'] = True
cong.head(3)

In [ ]:
cong['ingreso mensual'] = cong['total_ingreso'] / 12
cong['ingreso_menos_p'] = (cong['total_ingreso'] - cong['ingreso_publico'])/ 12

Otra forma es utilizando el ```eval```: 

In [ ]:
cong['new_ingreso_mp'] = cong.eval('(total_ingreso-ingreso_publico)/12')
cong['new_ingreso_mensual'] = cong.eval('total_ingreso/12')


Podemos crear variables categóricas en base a variables continuas con ```cut```: 

In [ ]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 10000, 50000, 100000, 200000000]

In [ ]:
cong2['cat_ingreso'] = pd.cut(cong2['total_ingreso'],
                              bins = categ_bins, labels = categ_labels)

Tarea: Crear cuantiles (qcut)

### Agregar datos

Muchas veces vamos a querer una agregación de los datos que nos resuma algunas de las variables en sus medidas de tendencia central, o las que nos interese. 

La forma más general de agregar los datos es la siguiente: 
    

In [ ]:
cong.median()

In [ ]:

cong['total_ingreso'].mean()

In [ ]:
cong[cong['partido'] == "PARTIDO MORADO"]['total_ingreso'].mean()

En general, podemos hacer todas estas agregaciones: 

* `mean` - promedio
* `median` - mediana
* `sum` - suma
* `min` - mínimo
* `max` - máximo
* `quantile` - nos da el cuantil que le pedimos
* `std` - desviación standar
* `var` - varianza
* `count` - produce el número de valores no missing
* `describe` - nos da varias de las medidas de arriba


### OJO: Podemos hacer agregaciones de ó columnas ó filas. 

Para hacer agregaciones por columnas: ```axis = 0``` ó ```axis = 'index' ```    
Para hacer agregaciones por filas: ```axis = 1``` ó ```axis = 'columns' ```

In [ ]:
## Sumando todos los ingresos por congresista
cong['otro_total'] = cong[['ingreso_publico',
       'ingreso_privado', 'renta_publico', 'renta_privado',
       'otro_ingreso_publico', 'otro_ingreso_privado']].sum(axis = 1)

In [ ]:
cong[['ingreso_publico',
       'ingreso_privado', 'renta_publico', 'renta_privado',
       'otro_ingreso_publico', 'otro_ingreso_privado']].quantile(q = 0.8, axis = 0)

In [ ]:
cong[['total_ingreso', 'partido']].groupby('partido').mean()

In [ ]:
(cong[['total_ingreso', 'partido']].groupby('partido').max().reset_index())

#.sort_values('total_ingreso')

Si bien hay varias formas, la más general es la siguiente:

In [ ]:
nueva_agregacion = cong.groupby(['partido']).agg(
{'total_ingreso': ['mean', 'max','min'], 
 'ingreso_privado': ['mean', 'median']}
)

In [ ]:
nueva_agregacion

In [ ]:
## El reset index es importante!
nueva_agregacion = cong.groupby(['partido']).agg(
{'total_ingreso': ['mean', 'max'], 
 'ingreso_privado': ['mean', 'median']}
).reset_index()



In [ ]:
nueva_agregacion

In [ ]:
nueva_agregacion.columns

In [ ]:
nueva_agregacion[(  'total_ingreso',   'mean')]

In [ ]:
nueva_agregacion.columns = [''.join(col).strip() for col in nueva_agregacion.columns.values]

In [ ]:
new_col = []
for col in nueva_agregacion.columns.values:
    a = ''.join(col).strip()
    new_col.append(a)
nueva_agregacion.columns = new_col

In [ ]:
nueva_agregacion


Una nueva forma de ordenar nuestro código es ponerlo entre paréntesis y separarlo por "puntitos"

In [ ]:
nueva_agregacion = (cong
                    .groupby(['partido'])
                    .agg(
                    {'total_ingreso': ['mean', 'max'], 
                     'ingreso_privado': ['mean', 'median']}
                   ).reset_index())



### Juntar datos (merge)

A veces tenemos los datos que nos interesan, esparcidos en diferentes dataframes.   
1er paso para hacer la unión de datos: Identificar el identificador único!



In [ ]:
cong_1 = cong[['dni', 'total_ingreso']]

cong_2 = cong[['dni', 'partido']]


La sintaxis más general de un merge es la siguiente: 

``` python
pd.merge(left = left_df,
         right=right_df,
         left_on=colname, 
         right_on=colname,
         how=merge_type)

```

El merge_type (o tipo de merge puede ser):

- left: quedan todas las observaciones (por keys) de las mano izquierda
- right: quedan todas las observaciones (por keys) de la mano derecha
- outer: es la unión de los keys de ambas bases
- inner: es la intersección de los keys de ambas bases. 


In [ ]:
resultado_merge = pd.merge(cong_1,
                     cong_2,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'inner', indicator = True)
resultado_merge

In [ ]:
solo_fp = cong_2.loc[cong2['partido'] == 'FUERZA POPULAR', :]

In [ ]:
resultado_merge_fp = pd.merge(cong_1,
                     solo_fp,
                    left_on = 'dni',
                    right_on = 'dni',
                    how = 'inner')
resultado_merge_fp